<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

# Read, Cache and Prefetch

In [2]:
batch_size = 32
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
val = val.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
test = test.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# Augmentation

In [3]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    # tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

# Visualisation

In [4]:
def summary_conv_and_last_dense(model):
    lines = []
    total_params = 0
    
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
            lines.append((layer.name, type(layer).__name__, layer.output.shape, layer.count_params()))
            total_params += layer.count_params()

    if lines:
        text = "----------------------------------------------------------------"
        text += f"\n{'Layer':<8}{'Output Shape':<20}{'Param #':>10}"
        text += "\n================================================================"
        for name, ltype, shape, params in lines:
            text += f"\n{ltype:<8}{str(shape):<20}{params:>10}"
        text += "\n================================================================"
        text += f"\nTotal params: {total_params}"
        print(text)
    else:
        print("No Conv2D or Dense layers found in the model.")

# Diff Evo and Func

In [5]:
def DE(pop_size, iterations, function, limits, ints):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    if ints:
        for it in ints:
            population[:, it] = np.round(population[:, it])

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    # function calls = pop_size + iterations * pop_size
    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            if ints:
                for it in ints:
                    mutant_vector[it] = np.round(mutant_vector[it])
            mutant_vector = np.clip(mutant_vector, x_low, x_high)
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    X = X.astype(int)
    amount_of_layer_1 = X[0]
    amount_of_layer_2 = X[1]
    amount_of_layer_3 = X[2]
    amount_of_one_dence = X[3]
    amount_of_one_filters = X[4]

    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters * 2), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters * 4), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(int(amount_of_one_filters * 8), activation='relu'))
    model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    # model.summary()
    # print(model.count_params())
    summary_conv_and_last_dense(model)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(train, batch_size=batch_size, epochs=100, validation_data=(val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    evaluation = model.evaluate(test)
    # print(model.metrics_names)
    # print(evaluation)
    loss, acc, top2  = evaluation
    # return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]
    return (1.0 - acc) + loss

func_limits = [[1, 4], [0, 4], [0, 4], [1, 4], [16, 64]]


# Use DE

In [6]:
best_f, best_d = DE(4, 4, func, func_limits, ints=[0, 1, 2, 3, 4])
best_f, best_d

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 26)         728
Conv2D  (None, 16, 16, 52)       12220
Conv2D  (None, 16, 16, 52)       24388
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 10)                2090
Total params: 213314
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 23ms/step - Top2: 0.3602 - accuracy: 0.1898 - loss: 2.1832 - val_Top2: 0.5176 - val_accuracy: 0.2819 - val_loss: 1.9114
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - Top2: 0.5064 - accuracy: 0.2789 - loss: 1.8998 - val_Top2: 0.5601 - val_accuracy: 0.3014 - val_loss: 1.7894
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - Top2: 0.5449 - accuracy: 0.3066 - loss: 1.8257 - val_Top2: 0.4634 - val_accuracy: 0.2651 - val_loss: 1.9447
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - Top

Processing:   0%|                                        | 0/4 [00:00<?, ?step/s]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 26)         728
Conv2D  (None, 16, 16, 52)       12220
Conv2D  (None, 16, 16, 52)       24388
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 208)              43472
Dense   (None, 10)                2090
Total params: 213314
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - Top2: 0.3646 - accuracy: 0.1921 - loss: 2.1731 - val_Top2: 0.4280 - val_accuracy: 0.2274 - val_loss: 2.0155
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - Top2: 0.5134 - accuracy: 0.2782 - loss: 1.9023 - val_Top2: 0.5668 - val_accuracy: 0.3197 - val_loss: 1.7820
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - Top2: 0.5577 - accuracy: 0.3204 - loss: 1.8151 - val_Top2: 0.4862 - val_accuracy: 0.2941 - val_loss: 1.8781
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - Top

Processing:  25%|██████████                              | 1/4 [2:12:36<6:37:49, 7956.38s/step]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 46)        1288
Conv2D  (None, 32, 32, 46)       19090
Conv2D  (None, 16, 16, 92)       38180
Conv2D  (None, 16, 16, 92)       76268
Dense   (None, 368)             135792
Dense   (None, 368)             135792
Dense   (None, 368)             135792
Dense   (None, 368)             135792
Dense   (None, 10)                3690
Total params: 681684
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 102s 79ms/step - Top2: 0.4047 - accuracy: 0.2159 - loss: 2.1228 - val_Top2: 0.5298 - val_accuracy: 0.2923 - val_loss: 1.8542
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 99s 79ms/step - Top2: 0.5475 - accuracy: 0.3160 - loss: 1.8339 - val_Top2: 0.5863 - val_accuracy: 0.3598 - val_loss: 1.7346
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 99s 79ms/step - Top2: 0.6019 - accuracy: 0.3736 - loss: 1.7175 - val_Top2: 0.5425 - val_accuracy: 0.3347 - val_loss: 1.8361
Epoch 4/100
1250/1250 

...

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 46)        1288
Conv2D  (None, 32, 32, 46)       19090
Conv2D  (None, 32, 32, 46)       19090
Conv2D  (None, 32, 32, 46)       19090
Conv2D  (None, 16, 16, 92)       38180
Conv2D  (None, 16, 16, 92)       76268
Conv2D  (None, 16, 16, 92)       76268
Conv2D  (None, 16, 16, 92)       76268
Dense   (None, 368)             135792
Dense   (None, 368)             135792
Dense   (None, 368)             135792
Dense   (None, 10)                3690
Total params: 736608
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 237s 186ms/step - Top2: 0.3975 - accuracy: 0.2166 - loss: 2.2168 - val_Top2: 0.4362 - val_accuracy: 0.2311 - val_loss: 2.0203
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 227s 182ms/step - Top2: 0.5431 - accuracy: 0.3122 - loss: 1.8332 - val_Top2: 0.5637 - val_accuracy: 0.3253 - val_loss: 1.9182
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 228s 182ms/step - Top2: 0.

Processing:  75%|██████████████████████████████          | 3/4 [12:20:34<4:33:13, 16393.79s/step]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 36)        1008
Conv2D  (None, 32, 32, 36)       11700
Conv2D  (None, 32, 32, 36)       11700
Conv2D  (None, 32, 32, 36)       11700
Conv2D  (None, 16, 16, 72)       23400
Conv2D  (None, 16, 16, 72)       46728
Conv2D  (None, 16, 16, 72)       46728
Conv2D  (None, 16, 16, 72)       46728
Dense   (None, 288)              83232
Dense   (None, 10)                2890
Total params: 285814
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 193s 151ms/step - Top2: 0.3475 - accuracy: 0.1848 - loss: 2.7659 - val_Top2: 0.4980 - val_accuracy: 0.2825 - val_loss: 1.9081
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 178s 143ms/step - Top2: 0.4777 - accuracy: 0.2519 - loss: 1.9543 - val_Top2: 0.5056 - val_accuracy: 0.2851 - val_loss: 1.9106
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 178s 142ms/step - Top2: 0.5327 - accuracy: 0.2916 - loss: 1.8656 - val_Top2: 0.4587 - val_accuracy: 0.26

Processing: 100%|████████████████████████████████████████| 4/4 [19:10:20<00:00, 17255.16s/step]  


(np.float64(0.9463974833488464), array([ 4.,  4.,  2.,  1., 38.]))

# Bests apart

In [7]:
best_f, best_d

(np.float64(0.9463974833488464), array([ 4.,  4.,  2.,  1., 38.]))

# Plains

## Func

In [8]:
def func(X):
    amount_of_layer_1 = 1
    amount_of_layer_2 = 1
    amount_of_layer_3 = 1
    amount_of_one_dence = 1
    amount_of_one_filters = int(round(X[0]))
    Dropout = X[1]
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(Dropout))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[16, 128], [0.01, 0.99]]

## Configuration

In [9]:
dots = 8
x1_lim = func_limits[0]
x1 = np.linspace(x1_lim[0], x1_lim[1], dots)
x2_lim = func_limits[1]
x2 = np.linspace(x2_lim[0], x2_lim[1], dots)
space = np.meshgrid(x1, x2)

In [10]:
fitness = np.array([[func(np.array([i, j]))] for j in x2] for i in x1)

## Plotting

## Thrash

In [11]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [12]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)